# Analysis

In this notebook we are going to explore different techniques to analyse our dataset and how we can
The first thing that we need to do is to import all the libraries that we are going to use.


## Set Up

For this second Notebook we need:

- **pandas**: The pandas module provides support to work with datasets e.g. data cleaning andand transformation.
- **nltk**: The NLTK library allows you to work with human language data (natural language processing). It includes libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning
- **numpy**: The numpy module is for scientific computing with Python. This will help with performing linear algebra and mathematic operations on your dataset.
- **matplotlib**: The matplotlib module will allow you to create visualisations.
- **seaborn**: The matplotlib module will allow you to create visualisations
- **wordcloud**: The wordcloud module will allow you to create wordcloud visualisations.
- **ast**: Ast is a powerful tool for interacting with and modifying Python code at a structural level.
- **collections** The collections library in Python offers specialized container datatypes that enhance the functionality, efficiency, and ease of use beyond Python's built-in containers like lists, dictionaries, and tuples.The defaultdict is a subclass of the standard Python dictionary and allows specifying a default value for missing keys

In [ ]:
import ast
from collections import defaultdict
import pandas as pd # We are renaming pandas as pd to be faster in calling it
import nltk
nltk.download('punkt_tab')# within nltk we need to download or import a series of submodules. Punkt is used for tokenizing sentences
nltk.download('stopwords')
from nltk.tokenize import word_tokenize # This is a specific import from the nltk.tokenize module, bringing in the word_tokenize function, which is used to split text into words.
from nltk.probability import FreqDist
from nltk.text import Text # need for keyword in context
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In theory you should still have the Parish object in your environment but if it passed some time between working on the first notebook and this notebook the kernel could have restarted.

Without having to do the cleaning and pre-processing again we can import the file directly from GitHub.

In [ ]:
# URL of the raw CSV file
url = "https://github.com/DCS-training/StatAccountScotland/blob/main/ParishTokenised.csv?raw=true"

# Use pandas to read the CSV file directly from the URL
Parish = pd.read_csv(url)
# Convert the 'Wordtokens' column from string to actual list
Parish['Wordtokens'] = Parish['Wordtokens'].apply(ast.literal_eval) #here we are using that ast module we mentioned above to make sure variable are recording properly

Parish.head()

Good! Now that we have done all the hard cleaning and pre-processing bit, let's start having a look at the analysis we can perform.

## 1 Keyword Frequency
The first type of analysis we are going to look at is the frequency of keyword.

Within this topic we are going to explore the frequency distribution of keywords
and method to visualise it (e.g. Wordclouds). Then we are going to look at the Term Frequency-Inverse Document Frequency or TF-IDF. TF-IDF is a numerical statistic used to indicate how important a word is to a document in a collection or corpus, adjusting the term frequency by how common the word is across all documents.

### 1a Frequency Distribution:
This is the most direct method in NLTK to handle keyword frequency. You can use the FreqDist class from NLTK to create a frequency distribution, which is essentially a count of each vocabulary item in the text.

The first thing we need to do is to flat our token to a single list of tokens across the whole dataset

In [ ]:
Tokens= Parish['Wordtokens'].tolist()
Tokens[0:15]
# Flatten the list of lists 'Tokens'
flat_tokens = [token for sublist in Tokens for token in sublist]
flat_tokens[0:15]

In [ ]:
fdist = FreqDist(flat_tokens)
print(fdist)

**Samples**: Refers to the unique tokens (or items) that were counted by FreqDist. In your case, "132623 samples" means there are 132,623 unique words or tokens in the dataset you analyzed.

**Outcomes**: Refers to the total number of tokens (or events, items, etc.) that were processed — essentially, the total count of all occurrences of all tokens in the data. "4697485 outcomes" indicates that, summing the counts of all individual tokens, there were 4,697,485 tokens processed in total

In [ ]:
# Plot the top 20 most common tokens
fdist.plot(20, title='Top 20 Most Common Tokens')
plt.show()

### 1b Wordcloud
A word cloud (also known as a tag cloud or word art) is a visual representation of text data where the size of each word indicates its frequency or importance within a large body of text.

In [ ]:
# Convert FreqDist to a dictionary for word cloud generation
freq_dict = dict(fdist)

In [ ]:
# Create the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(freq_dict)

# Display the word cloud
fig, ax = plt.subplots(figsize=(14, 7))  # Define the figure
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis('off')  # Turn off axis

In [ ]:
# Save the figure to a file
fig.savefig('wordcloud.jpg', format='jpeg')

Ok this is on the full dataset but it would probably more interesting to compare two areas on the dataset e.g one very city-like and one very remote

In [ ]:
print(Parish['Area'].unique())

In [ ]:
# Subsetting data for Edinburgh
edinburgh_tokens = Parish[Parish['Area'] == 'Edinburgh']['Wordtokens'].explode().tolist()

# Subsetting data for Fife
fife_tokens = Parish[Parish['Area'] == 'Fife']['Wordtokens'].explode().tolist()

In [ ]:
# Generate frequency distributions
freq_dist_edinburgh = FreqDist(edinburgh_tokens)
freq_dist_fife = FreqDist(fife_tokens)

In [ ]:
# to avoid having to repeat the steps multiple time we can create a function that will contain the specificsof creating and saving the visualisations

def create_plots(freq_dist, area_name):
    # Creating the frequency plot
    plt.figure(figsize=(10, 5))
    freq_dist.plot(30, title=f'Top 30 Most Frequent Words in {area_name}')
    plt.savefig(f'freq_plot_{area_name.lower()}.jpg')  # Save the frequency plot as a JPEG

    # Creating the word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(freq_dist)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Word Cloud for {area_name}')
    plt.savefig(f'wordcloud_{area_name.lower()}.jpg')  # Save the word cloud as a JPEG

# Create and save plots for Edinburgh
create_plots(freq_dist_edinburgh, 'Edinburgh')

# Create and save plots for Fife
create_plots(freq_dist_fife, 'Fife')

Can you see any big difference? Why do you think you are getting quite different results?

## Exercise 1

Try repeat the steps above for a different couple of areas.

**Tip:** we can reuse the function we created to create the graph


In [ ]:
# enter your code below

# Subsetting data for an area
#_tokens = Parish[Parish['Area'] == '_']['Wordtokens'].explode().tolist()


### 1c Term Frequency-Inverse Document Frequency
![TF-IDF.png](https://miro.medium.com/v2/resize:fit:720/format:webp/0*a-1asDQprowFNGIe.png)

From: https://nachi-keta.medium.com/nlp-what-is-tf-idf-149bc6a1da78

TF-IDF stands for Term Frequency-Inverse Document Frequency.
It reflects the importance of a word in a document within a collection or corpus.

TF-IDF takes into account both the frequency of a word within a document (term frequency) and the rarity of the word across the entire corpus (inverse document frequency).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# We are doing this on a subset cause our dataset is too big
caithness_texts = Parish[Parish['Area'] == 'Caithness']['text'][:3]

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Generate the TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(caithness_texts)

# Create a DataFrame to view the TF-IDF data clearly
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

print(df_tfidf)

# To see the IDF for each word
print("\nIDF for each word:")
for word, idf in zip(vectorizer.get_feature_names_out(), vectorizer.idf_):
    print(f"{word}: {idf}")



This is very complicated to decode giving how large our dataset still is.

Let's have a try on an artificial smaller dataset so it will be easier to understand the principles.

In [ ]:
# Generating sample documents
np.random.seed(0)  # For reproducible results

# Basic words list (excluding the stop words)
words = ["apple", "banana", "carrot", "date", "eggplant", "fig", "grape"]
# Stop words that will appear in every document
stop_words = ["the", "and", "is"]

# Generate documents with each having a mix of randomly selected words and all stop words
docs = [" ".join(list(np.random.choice(words, size=np.random.randint(2, 4), replace=False)) + stop_words)
        for _ in range(30)]
# Here's what some of the documents look like
print(docs[:5])


In [ ]:
# Compute the TF-IDF scores.
# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Generate the TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(docs)

# Create a DataFrame to view the TF-IDF data clearly
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())


In [ ]:
# Create a heatmap for the TF-IDF DataFrame
plt.figure(figsize=(12, 10))  # Set the figure size for better visibility
# Create the heatmap using seaborn
sns.heatmap(df_tfidf,# 'df_tfidf' is a DataFrame where columns correspond to terms and rows to documents
            annot=False,# 'annot=False' means we do not annotate each cell with numerical values to avoid clutter
            cmap="viridis",# 'cmap="viridis"' sets the color map to 'viridis', which is a perceptually uniform colormap
            cbar=True, # 'cbar=True' enables the color bar on the side, showing the scale of the TF-IDF values
            xticklabels=vectorizer.get_feature_names_out(), # 'xticklabels=vectorizer.get_feature_names_out()' uses the terms as labels for the x-axis
            linewidths=0.5, linecolor='black')# 'linewidths' and 'linecolor' are used to add borders to the cells

# Set the title of the heatmap
plt.title('TF-IDF Heatmap for 30 Documents')

# Set the label for the x-axis
plt.xlabel('Terms')

# Set the label for the y-axis
plt.ylabel('Document Index')

# Display the plot
plt.show()

## Exercise 2
Using the code below explore the high and low values for each term what is telling us?

In [ ]:
docs[14]
#docs[23]
#docs[24]

## 2 Keywords in Context
Keywords in context (KWIC) is a method of text analysis that involves displaying words from a specified text within the context of their surrounding words in a fixed number of characters or words. This approach, often used in concordance tools, helps in understanding how specific terms are used within a text, revealing patterns of usage and the semantic environments in which the keywords appear, aiding linguists and researchers in qualitative textual analysis.

In [ ]:
# Transforming the total of the token into a Text object
text_content = Text(flat_tokens)
# a "Text" object is a wrapper around a list of tokens (words), providing convenient methods to explore the text.

**NB** You have probably noticed by now tha different analyses and libraries require to manipulate to dataset to different formats. The wrong format of data is one of the most common reason why you get errors when try to apply functions to your dataset. Each library/function will have a page online that would describe the shape your dataset needs to have. So if in doubt google it.

### 2.1 Look at keywords in Context
The first and easier step is to select a keyword you are interesting in exploring and see what is present in the text around it

In [ ]:
# Let's work for example at where intemperance is mentioned
text_content.concordance('intemperance', lines=25)

Ok this is interesting in itself but we will have to look case by case.

Can we actually look at more recurring words around our keyword? i.e what are the most likely words to be found around intemperance? Unsurprisingly this will require yet some more data wrangling.


In [ ]:
# Fetching concordance lines which you can analyze or extract text from
concordance_lines = text_content.concordance_list("intemperance")

# Correctly extracting and concatenating text from each ConcordanceLine object
# Skipping the keyword itself because it will skew our analysis
contexts = [' '.join(line.left + line.right) for line in concordance_lines]# the ' ' is fundamental or your words will be all squished together

# Tokenize each context
tokenized_contexts = [word_tokenize(context) for context in contexts]

Let's look at what we have created

In [ ]:
print(tokenized_contexts[:2])

This is still a nested list so we need to flat them


In [ ]:
flat_tokens_intemperance = [token for sublist in tokenized_contexts for token in sublist]

Now we can finally plot

In [ ]:
freq_dist = FreqDist(flat_tokens_intemperance)
# Plot the top 20 most common tokens
freq_dist.plot(20, title='Top 20 Most Common Tokens near intemperance')
plt.show()

### 2.2 Similar tokens

We can also check similar token in context. Using similar(token) returns a list of words that appear in the same context as token.

In this case the the context is just the words directly on either side of token

In [ ]:
# First let's look at a new keyword == Church
text_content.concordance('church', lines=20)

In [ ]:
text_content.similar("church")

Using similar(token) returns a list of words that appear in the same context as token. In this case the the context is just the words directly on either side of token. So the list above is the list of words that has similar pre and after tokens to church e.g. "ridge manse situate".

We can even check what these concordance words are for each couple of words.

In [ ]:
text_content.common_contexts(['church', 'manse'])

## Exercise 3
What do you think this is telling us about the similarities between church and manse in our text?

## 3 Keyword Dispersion

We can also see how specific keywords re-occour across our dataset.
This is much more informative when you are analysing books (e.g. how often a carachters is named across the book) but it can be useful in our case too to look at specific keywords.

In [ ]:
text_content.dispersion_plot(["church", "sheep", "intemperance"])

In [ ]:
from nltk.corpus import gutenberg
nltk.download('gutenberg')
# Load the text of Moby Dick from the NLTK Gutenberg corpus
moby_dick_text = nltk.corpus.gutenberg.raw('melville-moby_dick.txt')

# Tokenize the text
tokens = nltk.word_tokenize(moby_dick_text)

# Create a Text object
text_object = Text(tokens)

# List of specific words to create a dispersion plot for
target_words = ['Ahab', 'whale', 'ship', 'sea', 'Ishmael']

# Generate the dispersion plot
text_object.dispersion_plot(target_words)

## 4 Bi-grams, N-Grams
When we talk about the context of words (or tokens!) in text analysis, we're referring to the surrounding words of a given word. Concordances show a bit of context to the left of an input word (just before the word appears) and to the right of that word (just after that word appeared).

We can use the similar method to see words that appear in similar contexts, meaning they're surrounded by similar tokens, as the token we input.

Pairs of words that occur together, such as "good" and "opinion," are referred to as **bigrams**, where "bi" indicates two. **N-grams** are groups of words that occur together, where n is a number of your choice.

To get all the bigrams in a text, we can use the bigrams() method, into which we pass the variable referring to the text itself.

Bigrams allow us to see which words commonly co-occur within a given dataset, which can be particularly useful

In [ ]:
bigrams_list = list(nltk.bigrams(text_content))
print(bigrams_list[:30])

In [ ]:
trigrams_list = list(nltk.trigrams(text_content))
print(trigrams_list[:30])

## 5 Working with Geographical Data
This is the last topic we are going to have a look.

Our dataset contains geographical information so we can also analyse our data based on their geographical location.
To do so we need a geographical file, in our case a geopackage file, (basically a vectorial file representing each area of Scotland). This file has been compiled in a way to match the areas in our dataset with the areas of historical parishes.

In [ ]:
#! pip install geopandas

### Set Up
Fot this last bit we need some additional packages that we haven't look yet.
- **geopandas** : is similar to Pandas but specifically for geographical dataset (that will have different file extensions).
- **matlibplot**: we have already import matlibplot but to print out geographical data we need some additional features that we need call separately
- **io, request and PIL**: you will need these additonal packages to be able to read and import an image hosted on a website. The first one will help with the encoding, the second with request data from the website, and the last with the image itself.

In [ ]:
import geopandas as gpd
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox # Import necessary classes
from io import BytesIO
import requests
from PIL import Image

In [ ]:
# import hte .gpkg file
url2 = "https://github.com/DCS-training/StatAccountScotland/blob/main/Spatial/Parishes.gpkg?raw=true"
# Adjust the path to where your GeoPackage file is located
ParishesGeo = gpd.read_file(url2, layer='civilparish_pre1891')

Let's say that we want to focus on the mentions of ilness in our dataset. First we need to search all mention of our ilness-related keyword and create a new column in our dataset that would print either yes or no if one of our keyword is present

In [ ]:
# if the beleow throw you an error because you do not have the Parish object anymore we can re-import it
# URL of the raw CSV file
#url = "https://raw.githubusercontent.com/DCS-training/StatAccountScotland/refs/heads/main/Parish.csv"
# Use pandas to read the CSV file directly from the URL
#Parish = pd.read_csv(url)
#Parish.info()

In [ ]:
# Check for keywords and add them to a new column
Parish['Illness'] = np.where(Parish['text'].str.contains('ill|illness|sick|cholera', case=False, regex=True), "yes", "no")

# Aggregate data by Area - Group by the Illness column and geographical area
IllnessGroup = Parish.groupby('Area').agg(
    Total=('Illness', 'size'),
    Count=('Illness', lambda x: np.sum(x == 'yes'))
).reset_index()
IllnessGroup['Percentage'] = round(IllnessGroup['Count'] / IllnessGroup['Total'], 2)

In [ ]:
# Perform the merge - ensuring similar keys are used for merging
MergedIlness = ParishesGeo.merge(IllnessGroup, left_on='JOIN_NAME_', right_on='Area', how='left')

In [ ]:
# Create a color map
color_palette = LinearSegmentedColormap.from_list("my_palette", ["white", "red"])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
MergedIlness.plot(column='Percentage',# that to use for colouring areas
                  cmap=color_palette, # using the palette I created
                  linewidth=0.8, # thinkness of the line
               ax=ax,
                  edgecolor='0',# colour of the lines
                  missing_kwds={'color': 'lightgrey'})# how to deal with missing values
ax.title.set_text('Illness Report Across Scotland')# title of the plot
plt.show()

we can also add a legend

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
divider = make_axes_locatable(ax)# to see the axes
cax = divider.append_axes("right", size="5%", pad=0.1)# cax = coloir axis. Create the legend


MergedIlness.plot(column='Percentage',
                  cmap=color_palette,
                  linewidth=0.8,
               ax=ax,
                  edgecolor='0',
                  missing_kwds={
                   'color': 'lightgrey',
                   'label': 'No data'
               },
               legend=True, cax=cax)# cax = coloir axis

ax.title.set_text('Illness Report Across Scotland')
ax.set_axis_off()  # Optionally turn off the axis.
plt.show()

Ok, let's now search for witches but this time I also want a scale bar and a north arrow

Now let's repeat our steps again but with a different subject i.e. witches

In [ ]:
# Check for keywords and add them to a new column
Parish['Witchcraft'] = np.where(Parish['text'].str.contains('witch|spell|witches|enchantemt|magic|witchcraft', case=False, regex=True), "yes", "no")

# Aggregate data by Area - Group by the 'Witchcraft column and geographical area
WitchcraftGroup = Parish.groupby('Area').agg(
    Total=('Witchcraft', 'size'),
    Count=('Witchcraft', lambda x: np.sum(x == 'yes'))
).reset_index()
WitchcraftGroup['Percentage'] = round(WitchcraftGroup['Count'] / WitchcraftGroup['Total'], 2)

In [ ]:
# Perform the merge - ensuring similar keys are used for merging
MergedWitches = ParishesGeo.merge(WitchcraftGroup , left_on='JOIN_NAME_', right_on='Area', how='left')

In [ ]:
# Create a color map
color_palette2 = LinearSegmentedColormap.from_list("my_palette", ["white", "purple"])

In [ ]:
# to add the scale bar and north arrow I need a couple more features from matlibplot
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
divider = make_axes_locatable(ax)# to see the colour legend
cax = divider.append_axes("right", size="5%", pad=0.1)

MergedWitches.plot(
    column='Percentage',
    cmap=color_palette2,
    linewidth=0.8,
    ax=ax,
    edgecolor='0',
    legend=True,
    cax=cax,
    missing_kwds={
        'color': 'lightgrey',
        'label': 'No data'
    }
)

ax.set_title('Witches Report Across Scotland')
ax.set_axis_off()

# Add scale bar
scalebar = AnchoredSizeBar(ax.transData,
                           100000, '100 km', 'lower left',# the unit of measure is meter
                           pad=0.4,
                           color='black',
                           frameon=False,
                           size_vertical=4,
                           fontproperties=fm.FontProperties(size=12))
ax.add_artist(scalebar)

# Add North Arrow
x, y, arrow_length = 0.95, 0.1, 0.1
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=12, xycoords=ax.transAxes)

plt.show()


### Using multiple geographical files
This is all nice but can we bring together multiple spatial information (e.g. shape and points)? Yes we can build up on our visualisation.

To do so we need to look at a different subject i.e. whisky for which we have at our dispostion also a geographical file containing punctual locations of modern distilleries.

Basically what we are trying to do is to see if the modern distilleries are mostly located in places where more mentions of whisky and alchool were.

In [ ]:
# Let's now search for wisky mentions
# Check for keywords and add them to a new column
Parish['Wisky'] = np.where(Parish['text'].str.contains('illicit still|illicit distillery|drunk|intemperance|wisky|whisky|whiskey|whysky|alembic', case=False, regex=True), "yes", "no")

# Aggregate data by Area - Group by the 'Wisky column and geographical area
WiskyGroup = Parish.groupby('Area').agg(
    Total=('Wisky', 'size'),
    Count=('Wisky', lambda x: np.sum(x == 'yes'))
).reset_index()
WiskyGroup['Percentage'] = round(WiskyGroup['Count'] / WiskyGroup['Total'], 2)

In [ ]:
# Perform the merge - ensuring similar keys are used for merging
MergedWisky = ParishesGeo.merge(WiskyGroup , left_on='JOIN_NAME_', right_on='Area', how='left')

In [ ]:
# Create a color map
color_palette3 = LinearSegmentedColormap.from_list("my_palette", ["white", "brown"])

In [ ]:
# Plot it
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

MergedWisky.plot(
    column='Percentage',
    cmap=color_palette3,
    linewidth=0.8,
    ax=ax,
    edgecolor='black',
    legend=True,
    cax=cax,
    missing_kwds={
        'color': 'lightgrey',
        'label': 'No data'
    }
)

ax.set_title('Whisky Report Across Scotland')
ax.set_axis_off()

# Add scale bar
scalebar = AnchoredSizeBar(ax.transData,
                           100000, '100 km', 'lower left',
                           pad=0.4,
                           color='black',
                           frameon=False,
                           size_vertical=4,
                           fontproperties=fm.FontProperties(size=12))
ax.add_artist(scalebar)

# Add North Arrow
x, y, arrow_length = 0.95, 0.1, 0.1
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=12, xycoords=ax.transAxes)

plt.show()

In [ ]:
# Import the location of modern day distilleries
url3 = "https://github.com/DCS-training/StatAccountScotland/blob/main/Spatial/ScottishDistilleries.gpkg?raw=true"
PointsDistilleries = gpd.read_file(url3)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# Plot area data
MergedWisky.plot(
    column='Percentage',
    cmap=color_palette3,
    linewidth=0.8,
    ax=ax,
    edgecolor='black',
    legend=True,
    cax=cax,
    missing_kwds={
        'color': 'lightgrey',
        'label': 'No data'
    }
)

# Handling points with custom icons
# Fetch and load the image from the web
url = "https://raw.githubusercontent.com/DCS-training/StatAccountScotland/main/Spatial/bottle.png"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img = np.array(img)
imagebox = OffsetImage(img, zoom=0.1)  # Adjust zoom as necessary

for x, y in zip(PointsDistilleries.geometry.x, PointsDistilleries.geometry.y):
    ab = AnnotationBbox(imagebox, (x, y), frameon=False, pad=0.1, box_alignment=(0.5, 0.5))
    ax.add_artist(ab)



ax.set_title("Whisky Reports across Scotland", fontsize=15)

# Add scale bar
scalebar = AnchoredSizeBar(ax.transData,
                           100000, '100 km', 'lower left',
                           pad=0.4,
                           color='black',
                           frameon=False,
                           size_vertical=4,
                           fontproperties=fm.FontProperties(size=12))
ax.add_artist(scalebar)

# Add North Arrow
x, y, arrow_length = 0.95, 0.1, 0.1
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=12, xycoords=ax.transAxes)
# Disable axis
ax.set_axis_off()

plt.show()


## Final Reflection
Is the location of modern distilleries similar to the location of historical ones?
Why do you think it may be?